In [19]:
from domainer.config import DATA_TMP_DIR, DATA_DIR
from lxml.etree import HTML
import json
import requests

In [4]:
# raw_ef html aquired manually from https://www.joinef.com/portfolio
# as it is a temporary file automation was not deemed as important as iteration speed
with open(DATA_TMP_DIR / "raw_ef.html") as f:
    raw_ef = f.read()

tree = HTML(raw_ef)
tree

<Element html at 0x7fc7b417fc00>

In [8]:
# extracting company data
companies = []

for company in tree.xpath(".//div[@class='companies loadmore-container']/div"):
    data = {
        "name": company.xpath("./a/text()")[0],
        "ef_link": company.xpath("./a/@href")[0],
        "tags": company.xpath(".//a[contains(@class, 'categorytag')]/@href"),
        "tagline": company.xpath(".//div[@class='tile__description text-sml']/text()")[0],
    }
    companies.append(data)

len(companies)

451

In [7]:
companies[0]

{'name': 'CodeAnt AI',
 'ef_link': 'https://www.joinef.com/companies/codeant-ai/',
 'tags': ['https://www.joinef.com/industry/ai/',
  'https://www.joinef.com/industry/developer-tools/'],
 'tagline': 'AI to auto-fix bad code and security vulnerabilities.'}

In [10]:
with open(DATA_TMP_DIR / "raw_ef.json", "w") as f:
    json.dump(companies, f)

In [11]:
# Function to extract data from raw company page
def extract_extra_ef_info(html: str) -> dict:
    tree = HTML(html)
    data = {
        "website": tree.xpath("//a[contains(@class, 'pageheader__websitebtn')]/@href"),
        "description": [
            desc
            for text in tree.xpath("//div[contains(@class, 'company__longbio')]//*/text()")
            + tree.xpath("//div[contains(@class, 'company__smallbio')]//*/text()")
            if (desc := text.strip())
        ],
    }

    return data

In [17]:
# Looping through the companies and getting the data, might take a bit of time
# Since requests are performed sequentially (took around 1m 27s on my run)
final_companies = []

for company in companies:
    cookies = {
        'cookieyes-consent': 'consentid:TGRZeW1mSkQyeExVUVloUnhCeXhidUZBRVVmN3FaSkM,consent:no,action:yes,necessary:yes,functional:no,analytics:no,performance:no,advertisement:no,other:no',
        'visitor_id1032441': '59950924',
        'visitor_id1032441-hash': 'b559b102cf45a5f7a2c59fb7eccb31eb16eda5a2e1120abfa19143166a9aee4c1f93f28e0af2f32c85d4be65d6b1ca4e18eafade',
    }

    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        # 'cookie': 'cookieyes-consent=consentid:TGRZeW1mSkQyeExVUVloUnhCeXhidUZBRVVmN3FaSkM,consent:no,action:yes,necessary:yes,functional:no,analytics:no,performance:no,advertisement:no,other:no; visitor_id1032441=59950924; visitor_id1032441-hash=b559b102cf45a5f7a2c59fb7eccb31eb16eda5a2e1120abfa19143166a9aee4c1f93f28e0af2f32c85d4be65d6b1ca4e18eafade',
        'pragma': 'no-cache',
        'priority': 'u=0, i',
        'referer': 'https://www.joinef.com/portfolio/?pagenum=3',
        'sec-ch-ua': '"Chromium";v="129", "Not=A?Brand";v="8"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Linux"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'sec-gpc': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    }
    
    response = requests.get(
        company["ef_link"], cookies=cookies, headers=headers
    )
    extra_data = extract_extra_ef_info(response.text)

    final_companies.append(company | extra_data)

final_companies[0]

{'name': 'CodeAnt AI',
 'ef_link': 'https://www.joinef.com/companies/codeant-ai/',
 'tags': ['https://www.joinef.com/industry/ai/',
  'https://www.joinef.com/industry/developer-tools/'],
 'tagline': 'AI to auto-fix bad code and security vulnerabilities.',
 'website': ['https://www.codeant.ai/'],
 'description': ['CodeAnt AI has developed a',
  'line-by-line code reviewer',
  'that saves developers time, and targets security vulnerabilities.',
  'Their AI Code Reviewer detects and auto-fixes code quality issues, highlighting best practices and anti-pattern violations for',
  'over 30 languages.',
  'Using an integration from developers’ IDEs to Pull Requests, CodeAnt AI’s code-quality tool acts as a',
  'clean code enforcer,',
  'ensuring bad code is never pushed again.']}

In [20]:
with open(DATA_DIR / "ef_final.json", "w") as f:
    json.dump(final_companies, f)